# Workbook for Final Cipher Challenge


In [32]:
:l Ciphers
{-# LANGUAGE ImportQualifiedPost #-}
import Ciphers qualified
import Ciphers.SubstitutionCipher
import Ciphers.SubstitutionCipher qualified as SC
import Ciphers.AbstractAlgebra qualified as Abs
import Ciphers.Common qualified as Common
import Ciphers.RSA qualified as RSA
import Ciphers.ShiftCipher qualified as ShiftCipher
import Ciphers.VigenereCipher qualified as VigenereCipher
import Ciphers.AffineCipher qualified as AffineCipher

import Control.Monad (forM_)
import Data.Foldable (foldl')
import Text.Printf (printf)
import Data.Char(isDigit)

# 1. Substitution Cipher

In [2]:
-- preview ciphertext
ciphertext = cp
print' ciphertext

OVDKLJOODKBPLABZGUFVDKLACVQLKBZVWLJJVGVLYBZVZUJJQLKBZVJLYVSUYVDTL
FVZVKDYOBZVJLRDJLYVCIZLJLFVZVKGUFVDKLACVGUFVDKLACVIUBZDIUJJQLKBZV
CLYCLQLJOODKBPLABZQLKBZVODAGZBVKCLQODKBPLABZBZLAGZKLAYOBZVGUKOJVO
VDKBZBZVRKLDPZVKCSVJJLYBZVPKVPDUYCBZVRZDFVBZVCBUJJYLKBZUYBZVUKZVD
KBCBZVZUJJIUYOCUYBZVUKFVUYCDYOBZVGKDYUBVLQYVIZDPSCZUKVUYBZVUKPACW
JVCDYOBZVUKTKDUYCDZUYBQLKWUSZVKCUHUCBZVLKOVKLQQUFVPLOAJLSUCCPDJJ

In [3]:
-- view frequencies
frequencies ciphertext

['V': 12.853470437017995
,'Z': 10.025706940874036
,'L': 8.740359897172237
,'B': 8.483290488431876
,'K': 8.483290488431876
,'U': 7.455012853470437
,'D': 6.169665809768637
,'J': 5.912596401028278
,'C': 5.3984575835475574
,'Y': 5.3984575835475574
,'O': 4.113110539845758
,'A': 2.827763496143959
,'P': 2.570694087403599
,'Q': 2.570694087403599
,'F': 2.056555269922879
,'G': 2.056555269922879
,'I': 1.2853470437017995
,'S': 1.2853470437017995
,'R': 0.7712082262210797
,'W': 0.7712082262210797
,'T': 0.5141388174807198
,'H': 0.2570694087403599
]

In [4]:
-- view tri-gram frequencies
mapM_ (putStr . show) $ take 10 $ frequencies $ ngrams 3 ciphertext

"BZV": 4.651162790697675
"KBZ": 1.550387596899225
"VDK": 1.550387596899225
"DKB": 1.2919896640826873
"LKB": 1.2919896640826873
"QLK": 1.2919896640826873
"YBZ": 1.2919896640826873
"DKL": 1.0335917312661498
"KLA": 1.0335917312661498
"OBZ": 1.0335917312661498

In [5]:
-- replace "BZV" with "the"
ciphertext1 = replace "BZV" "the" ciphertext
print' ciphertext1

OeDKLJOODKtPLAthGUFeDKLACeQLKtheWLJJeGeLYthehUJJQLKtheJLYeSUYeDTL
FeheKDYOtheJLRDJLYeCIhLJLFeheKGUFeDKLACeGUFeDKLACeIUthDIUJJQLKthe
CLYCLQLJOODKtPLAthQLKtheODAGhteKCLQODKtPLAththLAGhKLAYOtheGUKOJeO
eDKththeRKLDPheKCSeJJLYthePKePDUYCtheRhDFetheCtUJJYLKthUYtheUKheD
KtCthehUJJIUYOCUYtheUKFeUYCDYOtheGKDYUteLQYeIhDPSChUKeUYtheUKPACW
JeCDYOtheUKTKDUYCDhUYtQLKWUSheKCUHUCtheLKOeKLQQUFePLOAJLSUCCPDJJ

In [6]:
-- we can expect 'L' to map to either 'a' or 'o'
ciphertext2a = replace "L" "a" ciphertext1
ciphertext2b = replace "L" "o" ciphertext1

print' ciphertext2a
putStrLn "-----------------------------------------------------------------"
print' ciphertext2b



OeDKaJOODKtPaAthGUFeDKaACeQaKtheWaJJeGeaYthehUJJQaKtheJaYeSUYeDTa
FeheKDYOtheJaRDJaYeCIhaJaFeheKGUFeDKaACeGUFeDKaACeIUthDIUJJQaKthe
CaYCaQaJOODKtPaAthQaKtheODAGhteKCaQODKtPaAththaAGhKaAYOtheGUKOJeO
eDKththeRKaDPheKCSeJJaYthePKePDUYCtheRhDFetheCtUJJYaKthUYtheUKheD
KtCthehUJJIUYOCUYtheUKFeUYCDYOtheGKDYUteaQYeIhDPSChUKeUYtheUKPACW
JeCDYOtheUKTKDUYCDhUYtQaKWUSheKCUHUCtheaKOeKaQQUFePaOAJaSUCCPDJJ

-----------------------------------------------------------------

OeDKoJOODKtPoAthGUFeDKoACeQoKtheWoJJeGeoYthehUJJQoKtheJoYeSUYeDTo
FeheKDYOtheJoRDJoYeCIhoJoFeheKGUFeDKoACeGUFeDKoACeIUthDIUJJQoKthe
CoYCoQoJOODKtPoAthQoKtheODAGhteKCoQODKtPoAththoAGhKoAYOtheGUKOJeO
eDKththeRKoDPheKCSeJJoYthePKePDUYCtheRhDFetheCtUJJYoKthUYtheUKheD
KtCthehUJJIUYOCUYtheUKFeUYCDYOtheGKDYUteoQYeIhDPSChUKeUYtheUKPACW
JeCDYOtheUKTKDUYCDhUYtQoKWUSheKCUHUCtheoKOeKoQQUFePoOAJoSUCCPDJJ

In [7]:
-- view bi-gram frequencies
mapM_ (putStr . show) $ take 10 $ frequencies $ ngrams 4 ciphertext2b

"Ythe": 1.2953367875647668
"oKth": 1.2953367875647668
"Kthe": 1.0362694300518134
"Othe": 1.0362694300518134
"QoKt": 1.0362694300518134
"YOth": 1.0362694300518134
"eDKo": 1.0362694300518134
"heUK": 1.0362694300518134
"theU": 1.0362694300518134
"Cthe": 0.7772020725388601

In [8]:
--- we see "oKth" as one of the most frequent four-grams. We can replace "K" with "r"
ciphertext3 = replace "K" "r" ciphertext2b
print' ciphertext3

OeDroJOODrtPoAthGUFeDroACeQortheWoJJeGeoYthehUJJQortheJoYeSUYeDTo
FeherDYOtheJoRDJoYeCIhoJoFeherGUFeDroACeGUFeDroACeIUthDIUJJQorthe
CoYCoQoJOODrtPoAthQortheODAGhterCoQODrtPoAththoAGhroAYOtheGUrOJeO
eDrththeRroDPherCSeJJoYthePrePDUYCtheRhDFetheCtUJJYorthUYtheUrheD
rtCthehUJJIUYOCUYtheUrFeUYCDYOtheGrDYUteoQYeIhDPSChUreUYtheUrPACW
JeCDYOtheUrTrDUYCDhUYtQorWUSherCUHUCtheorOeroQQUFePoOAJoSUCCPDJJ

In [9]:
--
mapM_ (putStr . show) $ take 10 $ frequencies $ ngrams 3 ciphertext3

"the": 4.651162790697675
"eDr": 1.550387596899225
"rth": 1.550387596899225
"Drt": 1.2919896640826873
"Qor": 1.2919896640826873
"Yth": 1.2919896640826873
"ort": 1.2919896640826873
"Dro": 1.0335917312661498
"Oth": 1.0335917312661498
"UFe": 1.0335917312661498

In [10]:
ciphertext4 = replace "D" "a" ciphertext3
print' ciphertext4

OearoJOOartPoAthGUFearoACeQortheWoJJeGeoYthehUJJQortheJoYeSUYeaTo
FeheraYOtheJoRaJoYeCIhoJoFeherGUFearoACeGUFearoACeIUthaIUJJQorthe
CoYCoQoJOOartPoAthQortheOaAGhterCoQOartPoAththoAGhroAYOtheGUrOJeO
earththeRroaPherCSeJJoYthePrePaUYCtheRhaFetheCtUJJYorthUYtheUrhea
rtCthehUJJIUYOCUYtheUrFeUYCaYOtheGraYUteoQYeIhaPSChUreUYtheUrPACW
JeCaYOtheUrTraUYCahUYtQorWUSherCUHUCtheorOeroQQUFePoOAJoSUCCPaJJ

In [11]:
ciphertext5 = replace "O" "d" ciphertext4
print' ciphertext5

dearoJddartPoAthGUFearoACeQortheWoJJeGeoYthehUJJQortheJoYeSUYeaTo
FeheraYdtheJoRaJoYeCIhoJoFeherGUFearoACeGUFearoACeIUthaIUJJQorthe
CoYCoQoJddartPoAthQorthedaAGhterCoQdartPoAththoAGhroAYdtheGUrdJed
earththeRroaPherCSeJJoYthePrePaUYCtheRhaFetheCtUJJYorthUYtheUrhea
rtCthehUJJIUYdCUYtheUrFeUYCaYdtheGraYUteoQYeIhaPSChUreUYtheUrPACW
JeCaYdtheUrTraUYCahUYtQorWUSherCUHUCtheorderoQQUFePodAJoSUCCPaJJ

In [12]:
ciphertext6 = replace "J" "l" ciphertext5
print' ciphertext6

dearolddartPoAthGUFearoACeQortheWolleGeoYthehUllQortheloYeSUYeaTo
FeheraYdtheloRaloYeCIholoFeherGUFearoACeGUFearoACeIUthaIUllQorthe
CoYCoQolddartPoAthQorthedaAGhterCoQdartPoAththoAGhroAYdtheGUrdled
earththeRroaPherCSelloYthePrePaUYCtheRhaFetheCtUllYorthUYtheUrhea
rtCthehUllIUYdCUYtheUrFeUYCaYdtheGraYUteoQYeIhaPSChUreUYtheUrPACW
leCaYdtheUrTraUYCahUYtQorWUSherCUHUCtheorderoQQUFePodAloSUCCPall

In [13]:
ciphertext7 = replace "Q" "f" ciphertext6
print' ciphertext7

dearolddartPoAthGUFearoACefortheWolleGeoYthehUllfortheloYeSUYeaTo
FeheraYdtheloRaloYeCIholoFeherGUFearoACeGUFearoACeIUthaIUllforthe
CoYCofolddartPoAthforthedaAGhterCofdartPoAththoAGhroAYdtheGUrdled
earththeRroaPherCSelloYthePrePaUYCtheRhaFetheCtUllYorthUYtheUrhea
rtCthehUllIUYdCUYtheUrFeUYCaYdtheGraYUteofYeIhaPSChUreUYtheUrPACW
leCaYdtheUrTraUYCahUYtforWUSherCUHUCtheorderoffUFePodAloSUCCPall

In [14]:
-- view bi-gram frequencies
mapM_ (putStr . show) $ take 10 $ frequencies $ ngrams 5 ciphertext7

"Ydthe": 1.0389610389610389
"forth": 1.0389610389610389
"orthe": 1.0389610389610389
"theUr": 1.0389610389610389
"Fearo": 0.7792207792207793
"GUFea": 0.7792207792207793
"PoAth": 0.7792207792207793
"UFear": 0.7792207792207793
"UYthe": 0.7792207792207793
"YtheU": 0.7792207792207793

In [15]:
ciphertext8 = replace "U" "i" ciphertext7
print' ciphertext8

dearolddartPoAthGiFearoACefortheWolleGeoYthehillfortheloYeSiYeaTo
FeheraYdtheloRaloYeCIholoFeherGiFearoACeGiFearoACeIithaIillforthe
CoYCofolddartPoAthforthedaAGhterCofdartPoAththoAGhroAYdtheGirdled
earththeRroaPherCSelloYthePrePaiYCtheRhaFetheCtillYorthiYtheirhea
rtCthehillIiYdCiYtheirFeiYCaYdtheGraYiteofYeIhaPSChireiYtheirPACW
leCaYdtheirTraiYCahiYtforWiSherCiHiCtheorderoffiFePodAloSiCCPall

In [16]:
-- view bi-gram frequencies
mapM_ (putStr . show) $ take 10 $ frequencies $ ngrams 3 ciphertext8

"the": 4.651162790697675
"ear": 1.550387596899225
"rth": 1.550387596899225
"Yth": 1.2919896640826873
"art": 1.2919896640826873
"for": 1.2919896640826873
"ort": 1.2919896640826873
"Ydt": 1.0335917312661498
"aro": 1.0335917312661498
"dth": 1.0335917312661498

In [17]:
ciphertext9 = replace "Q" "f" ciphertext8
print' ciphertext9

dearolddartPoAthGiFearoACefortheWolleGeoYthehillfortheloYeSiYeaTo
FeheraYdtheloRaloYeCIholoFeherGiFearoACeGiFearoACeIithaIillforthe
CoYCofolddartPoAthforthedaAGhterCofdartPoAththoAGhroAYdtheGirdled
earththeRroaPherCSelloYthePrePaiYCtheRhaFetheCtillYorthiYtheirhea
rtCthehillIiYdCiYtheirFeiYCaYdtheGraYiteofYeIhaPSChireiYtheirPACW
leCaYdtheirTraiYCahiYtforWiSherCiHiCtheorderoffiFePodAloSiCCPall

In [18]:
-- view bi-gram frequencies
mapM_ (putStr . show) $ take 10 $ frequencies $ ngrams 6 ciphertext9

"forthe": 1.0416666666666667
"FearoA": 0.78125
"GiFear": 0.78125
"Ytheir": 0.78125
"aYdthe": 0.78125
"aroACe": 0.78125
"artPoA": 0.78125
"dartPo": 0.78125
"earoAC": 0.78125
"iFearo": 0.78125

In [19]:
ciphertext10 = replace "WG" "cg" ciphertext9
print' ciphertext10

dearolddartPoAthgiFearoACeforthecollegeoYthehillfortheloYeSiYeaTo
FeheraYdtheloRaloYeCIholoFehergiFearoACegiFearoACeIithaIillforthe
CoYCofolddartPoAthforthedaAghterCofdartPoAththoAghroAYdthegirdled
earththeRroaPherCSelloYthePrePaiYCtheRhaFetheCtillYorthiYtheirhea
rtCthehillIiYdCiYtheirFeiYCaYdthegraYiteofYeIhaPSChireiYtheirPACc
leCaYdtheirTraiYCahiYtforciSherCiHiCtheorderoffiFePodAloSiCCPall

In [20]:
ciphertext11 = replace "YPA" "nmu" ciphertext10
print' ciphertext11

dearolddartmouthgiFearouCeforthecollegeonthehillfortheloneSineaTo
FeherandtheloRaloneCIholoFehergiFearouCegiFearouCeIithaIillforthe
ConCofolddartmouthforthedaughterCofdartmouththoughroundthegirdled
earththeRroamherCSellonthemremainCtheRhaFetheCtillnorthintheirhea
rtCthehillIindCintheirFeinCandthegraniteofneIhamSChireintheirmuCc
leCandtheirTrainCahintforciSherCiHiCtheorderoffiFemoduloSiCCmall

In [21]:
ciphertext12 = replace "FCISTRH" "vswpbyx" ciphertext11
print' ciphertext12

dearolddartmouthgivearouseforthecollegeonthehillforthelonepineabo
veherandtheloyaloneswholovehergivearousegivearousewithawillforthe
sonsofolddartmouthforthedaughtersofdartmouththoughroundthegirdled
earththeyroamherspellonthemremainstheyhavethestillnorthintheirhea
rtsthehillwindsintheirveinsandthegraniteofnewhampshireintheirmusc
lesandtheirbrainsahintforciphersixistheorderoffivemodulopissmall

# 2. Vigenere Cipher

In [54]:
-- ShiftCipher.bruteforce "johnconway"
print' VigenereCipher.cp

ciphertext = VigenereCipher.cp

decrypt = VigenereCipher.decrypt

ShiftCipher.shiftChar 'e' 'v'

ShiftCipher.shiftChar 'e' 'n'

decrypt ciphertext "johnconway"

hcbxpcjlemyzlgjwagtfjhtnvvriarrqzvuqbipjrqhggrzwtfnahgkqfesrqszvo
dyabgcwafvvrotsotdreoaqnbnfzgcbqetqloafvvnpapnqvzrzvyarnrpzgoashy
cwzvvwaphmbssvvhyammusjhnsfwnbbhbshhuwtkjylhrangemwsjnvprdatnrpsh
seanrumabcbbphcacygjogiainojnvvbsdmhcbqgtvjeyloavjoianmrrln

'Z'

'R'

"YOUKNOWPEOPLETHINKTHATMATHEMATICSISCOMPLICATEDMATHEMATICSISTHESIMPLEBITITSTHESTUFFWECANUNDERSTANDITSCATSTHATARECOMPLICATEDIMEANWHATISITINTHOSELITTLEMOLECULESANDSTUFFTHATMAKEUPMAKEONECATBEHAVEDIFFERENTLYTOANOTHERORTHATMAKEACATHOWDOYOUDEFINEACATIHAVENOIDEA"

# 3. Affine Cipher

In [23]:
ciphertext = [6917141364293641, 5044493105177484, 10208794241351887, 16394322558427148,
  11758121930809893, 15571898457877977, 7672722015089403, 13661070158473411,
  17999297470735005, 12313955920676335, 5960590266677512, 1613421779734456,
  1750819096862416, 3118598423638319, 14816640742963862, 4952241931583899,
  12257144082730227, 7862771476786858, 5006500927265261, 11323114722137903,
  22833602100630408, 8963415721169565, 15595638667025459, 8028339051359388,
  3385708046121353, 12190779082257523, 8983375210790796, 15571898457877977,
  15147654701575566, 16361132341028484, 5962327355151718, 8901193427034701,
  5179568152435730, 3672045789372412, 23610469115026974, 1577294047287513,
  15642317927380556, 15571898457877977, 10282634434851196, 10749617216933305,
  17838746455253440, 21499666401460178, 1037344909841996, 17413814796435480,
  16269186929768054, 10449344135634668, 24087490685235750, 10768725190149571,
  6484888204271905, 22185358129776042, 19377417029468988, 16267449841293848,
  16555381474675390, 21520574190817628, 14140526597210259, 19733309797334806,
  16283129124025650, 16538093542757166, 24098448719654436, 16798515250649044,
  13879801264995293, 10264930014031131, 7946076055449771, 18258106201941864,
  423054714981679, 17458353983971638, 9294184051519018, 19030921054252445]

c1 = 6917141364293641
c2 = 5044493105177484
c3 = 10208794241351887

x1 = 314077111660
x2 = 464400513312
x3 = 495875089509

xx = (x2 - x3) * (c1 - c2) - (x1 - x2) * (c2 - c3)
xx' = -xx
print xx'
-- Abs.primeFactors xx'

-- mapM_ (putStr . show) $ take 10 (Abs.primeFactors xx')
-- Abs.pollardRho xx'
-- Abs.pollardRho 167051224853351122452125737
-- Abs.pollardRho 988468786114503683148673
-- Abs.pollardRho 1980899370970949264827
-- Abs.pollardRho 24610808569754243

n = [167051224853351122452125737, 988468786114503683148673, 1980899370970949264827, 24610808569754243]

24610808569754243 > 10 * 10208794241351887
m :: [Integer]
m = map (round . (xx' /)) n
m

all = foldl' (&&) True


forM_ n $ \x -> do
  print x
  putStr "\n\n"
  let safe = all $ map (< x) ciphertext
  print safe

print $ 6917141364293641 - 5044493105177484
print $ 314077111660 - 464400513312

835256124266755612260628685

False

[5,845,421655,33938589295]

167051224853351122452125737


True
988468786114503683148673


True
1980899370970949264827


True
24610808569754243


True

1872648259116157

-150323401652

In [24]:
print $ Abs.invN 10 1

printf "n = %d" $ 24610808569754243 - 150323401652

base = 24610808569754243
n = 24610658246352591

printf "extended euclidean: %s" $ show $ Abs.extendedEuclidean base n

printf "x = %d" $ Abs.mulN base 2956863623047174 1872648259116157

printf "xy = %d" $ Abs.mulN base 289514745701 314077111660

printf "c - xy = %d" $ Abs.subN base 6917141364293641 6917141364293672

printf "inverse x = %s" $ show $ Abs.extendedEuclidean base 289514745701

1

n = 24610658246352591

extended euclidean: (1,-2956845562454131,2956863623047174)

x = 289514745701

xy = 6917141364293672

c - xy = 24610808569754212

inverse x = (1,-77853851833,6618130068783420)

In [25]:


sub = 24610808569754212
mul = 6618130068783420

-- decrypt

x = 10208794241351887
printf "simple decryption of %d = %d" x $ AffineCipher.decrypt base sub mul x

AffineCipher.decryptAll base sub mul ciphertext



simple decryption of 10208794241351887 = 495875089509

[314077111660,464400513312,495875089509,474400107552,147495347566,139476301088,444083740788,448378660128,457135256352,499967423520,521560989800,418598166626,435493412979,477284692585,465675313518,500036083828,491260571237,430040313714,189522408819,498760574317,435626861938,139391951220,139476301170,139358200172,139224637984,452903072872,418560108902,139476301088,418531057766,491327400992,521560989806,435492757620,477284954725,434336132973,435443164281,139140559717,198107482470,139476301088,448311747872,495790679328,482906432882,189523060837,472991231861,490170117986,139208106100,477284887920,478744506912,495790679394,478427029605,465792478752,138855082355,139208106094,478689651317,495869518112,444300616237,422540943459,418463906080,444016190766,146567877736,434333118057,444216713330,452823839604,435610744174,442925215602,139106807912,435706410016,452903062867,305716535328]

In [26]:
-- def strtoint(plaintext_block):
--   modn = [bin(ord(c))[2:] for c in plaintext_block]
--   modn = [ ''.join(['0' for i in range(8-len(m))]) + m for m in modn]
--   return int('0b' + ''.join(modn),2)

-- # inttostr is the inverse of strtoint; it takes as input an integer encoding of a string (ASCII bytes) and returns as output the string
-- def inttostr(int_block):
--   # print(int_block)
--   s = bin(int_block)[2:]
--   # print(s)
--   s = ''.join(['0' for i in range(8-(len(s)%8))]) + s
--   # print(s)
--   s = ['0b' + s[8*i:8*(i+1)] for i in range(len(s)//8)]
--   # print(s)
--   return ''.join([chr(int(c,2)) for c in s])

-- decrypted = [314077111660,464400513312,495875089509,474400107552,147495347566,139476301088,444083740788,448378660128,457135256352,499967423520,521560989800,418598166626,435493412979,477284692585,465675313518,500036083828,491260571237,430040313714,189522408819,498760574317,435626861938,139391951220,139476301170,139358200172,139224637984,452903072872,418560108902,139476301088,418531057766,491327400992,521560989806,435492757620,477284954725,434336132973,435443164281,139140559717,198107482470,139476301088,448311747872,495790679328,482906432882,189523060837,472991231861,490170117986,139208106100,477284887920,478744506912,495790679394,478427029605,465792478752,138855082355,139208106094,478689651317,495869518112,444300616237,422540943459,418463906080,444016190766,146567877736,434333118057,444216713330,452823839604,435610744174,442925215602,139106807912,435706410016,452903062867,305716535328]

-- text = [inttostr(i) for i in decrypted]

-- kx = 289514745701
-- ky = 24610808569754212
-- key = [kx, ky]
-- dkey = [inttostr(i) for i in key]
-- for i in text:
--   print(i, end = ' ')

-- print("\n")

-- for i in dkey:
--   print(i, end = ' ')

putStr "I tel l my  stude nts,  \"When  you  get t hese  jobs  that  you h ave b een s o bri llian tly t raine d for , jus t rem ember  that  your  real  job  is th at if  you  are f ree,  you n eed t o fre e som ebody  else .  If  you  have  some  power , the n you r job  is t o emp ower  someb ody e lse.   This  is n ot ju st a  grab- bag c andy  game. \"  Th e Eni gma r ingst ellun g for  ciph er 4  is MS G."

putStr "Chloe Wofford"

I tel l my  stude nts,  "When  you  get t hese  jobs  that  you h ave b een s o bri llian tly t raine d for , jus t rem ember  that  your  real  job  is th at if  you  are f ree,  you n eed t o fre e som ebody  else .  If  you  have  some  power , the n you r job  is t o emp ower  someb ody e lse.   This  is n ot ju st a  grab- bag c andy  game. "  Th e Eni gma r ingst ellun g for  ciph er 4  is MS G.

Chloe Wofford

# 5. RSA

In [27]:
n = 2490736346492104721729722567335076257528146493316778156981974373631622149367708642633012928856521206471732646282243373960731
e = 65537
f = 1000003
-- print $ Abs.extendedEuclidean n e
-- eA' = 379062275049395799236038465089949960977550591640470960491603405749451444493851198584336648800142553265316712910555799195635
-- print $ Abs.mulN n eA eA'
-- print $ Abs.extendedEuclidean n eB
-- eB' = 624809301035027332782789476537843605975147734991282093945274363699895842969958273964239932535576163999768021946420040071425
-- print $ Abs.mulN n eB eB'
mA = 1175930342106065271050283250310309419316488156305401520290443548602035808283262101734292122005367394797217259116903341956086
mB = 1799306147392696606782310867010621904452538401321735738023524507758249509686120401186634973441137240046185785005515614987681
-- dA = Abs.powN n mA eA'
-- dB = Abs.powN n mB eB'

-- printf "dA = %d" dA
-- printf "dB = %d" dB

-- print $ dB ^ eB
-- printf "phi n = $d" $ Abs.phi' n
-- printf "phi n = %d" $ Abs.phi n
-- Abs.fermi n
Abs.extendedEuclidean e f
-- printf "%d" $ eA * eB

print $ 295788 * e - 19385 * f

-- Abs.addN n 0 (-19385)

x = 295788
y = -19385

Abs.extendedEuclidean y n

-- print $ Abs.addN n (Abs.mulN n e * x) (f * y)

negP = Abs.powN n mB (-y)
printf "negP = %d" negP
(quot, a', b') = Abs.extendedEuclidean n negP
posP = b' -- -556401337761795780749873534229349540641947639134298692705685418372544768452480449926418213910000320707363825780743677422132

decrypted = ((Abs.powN n mA x) * posP) `mod` n
printf "decrypted = %d" decrypted

-- decrypted = 1583081514314960595845670857721062541401326299686994544103204912108729041871355509996620895411011543463268620160947108900639

print $ Abs.powN n decrypted e

print $ Abs.powN n decrypted f



Line 41: Redundant bracket
Found:
(Abs.powN n mA x) * posP
Why not:
Abs.powN n mA x * posP

(1,295788,-19385)

1

(-1,-38674833133563245872615243372084495925507974954261038186823538120356887643006463834538916254104353012535028451429211016878,-301)

negP = 574370848784842328679841258247477447928083174314548675875021027931091811294130798833804852599130583872677012881855238226892

decrypted = 148287856838141344109395422592587418669933303609979671462598256391365083697606202170869585351243553

1175930342106065271050283250310309419316488156305401520290443548602035808283262101734292122005367394797217259116903341956086

1799306147392696606782310867010621904452538401321735738023524507758249509686120401186634973441137240046185785005515614987681

In [28]:
def strtoint(plaintext_block):
  modn = [bin(ord(c))[2:] for c in plaintext_block]
  modn = [ ''.join(['0' for i in range(8-len(m))]) + m for m in modn]
  return int('0b' + ''.join(modn),2)

# inttostr is the inverse of strtoint; it takes as input an integer encoding of a string (ASCII bytes) and returns as output the string
def inttostr(int_block):
  # print(int_block)
  s = bin(int_block)[2:]
  # print(s)
  s = ''.join(['0' for i in range(8-(len(s)%8))]) + s
  # print(s)
  s = ['0b' + s[8*i:8*(i+1)] for i in range(len(s)//8)]
  # print(s)
  return ''.join([chr(int(c,2)) for c in s])


decrypted = 148287856838141344109395422592587418669933303609979671462598256391365083697606202170869585351243553

print(inttostr(decrypted))


: 

: 

: 

In [ ]:
-- # def convert(n, base):
-- #   highest_power = 0
-- #   while (base ** (highest_power + 1)) < n:
-- #     highest_power += 1

-- #   quotient = n // highest_power
-- #   if highest_power == 0:
-- #     return quotient
-- #   return quotient * 10 ^ highest_power + convert(highest_power - quotient, base)

-- # convert(26, 26)

In [45]:
n = 166530529432965340099271666821176530188535972283046096996016364234771423224878910478932117699610350618246947860042343417159
e = 65537
y = 149441402545605284082094630171037686837410554943035204389042797718540079485257006113787250895354916689260854817821121453874

-- Abs.pollardRho n
e' = 50827932011651520484699195132886676737740284931836536280441793727734881147087355424965028798253793847992240038067422171393

y' = Abs.powN n y e'
printf "y^e' = %d" y'
Abs.addN n 0 (-5780810144803639909148161222182532099103091947204325353097292043183316109016273575836101252218037866495442366626429822452)

y26 = "DE07I0OICKBJ8FBO8D10I4H08D1EM"
y26' = map (\x -> if isDigit x then Common.toInt x else 10 + (Common.charToInt x)) y26
print y26'

string = map (\x -> Common.chr (Common.ord 'A' + x)) y26'
print string

Line 13: Redundant bracket
Found:
10 + (Common.charToInt x)
Why not:
10 + Common.charToInt x

y^e' = 56342628068105285834414495251222356910958

160749719288161700190123505598993998089432880335841771642919072191588107115862636903096016447392312751751505493415913594707

[13,14,0,7,18,0,24,18,12,20,11,19,8,15,11,24,8,13,1,0,18,4,17,0,8,13,1,14,22]

"NOAHSAYSMULTIPLYINBASERAINBOW"

In [55]:
p = 2147483647
print $ sqrt p

46340.950001051984